In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl")
from travel import init_travel
init_travel()

/home/sstorks/.cache/pypoetry/virtualenvs/travel-nQET-zRt-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Load preds

In [2]:
import json

RESULTS_PATH = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_debug500/llava-1.5-7b-hf/VQG2VQA_ego4d_debug500_llava-1.5-7b-hf_spatial_norephrase1.0_20240713191244/preds_nli_val.json" # with NLI correction (v2)
# RESULTS_PATH = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d_debug500/llava-1.5-7b-hf/VQG2VQA_ego4d_debug500_llava-1.5-7b-hf_spatial_norephrase1.0_20240713091801/preds_nli_val.json" # without NLI correction
preds = json.load(open(RESULTS_PATH, "r"))

# Visualize some outputs

Randomly choose:

In [3]:
import random

output_indices_to_visualize = random.sample(list(range(len(preds))), 20)

print("Indices to visualize:", output_indices_to_visualize)

Indices to visualize: [398, 55, 120, 145, 156, 16, 14, 246, 114, 227, 378, 368, 225, 231, 38, 396, 8, 52, 479, 214]


Or choose based on conditions:

In [3]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image
from pprint import pprint
import random
from tqdm import tqdm

from travel.data.ego4d import Ego4DMistakeDetectionDataset
from travel.data.mistake_detection import MistakeDetectionExample, get_cutoff_time_by_proportion
from travel.model.vqa import VQAResponse
from travel.model.mistake_detection import aggregate_mistake_probs_over_frames
from travel.model.mistake_detection import DETECTION_FRAMES_PROPORTION

output_indices_to_visualize = []
for pred_idx, pred in enumerate(tqdm(preds.values())):
    frame_times = pred['example']['frame_times']
    cutoff_time = get_cutoff_time_by_proportion(frame_times, DETECTION_FRAMES_PROPORTION)
    frame_times = [t for t in frame_times if t >= cutoff_time]
    mistake_prob = aggregate_mistake_probs_over_frames(pred['mistake_detection']["0.0"]['mistake_probs'], frame_times)
    
    predicted_mistake = mistake_prob >= 0.5
    conf = mistake_prob if predicted_mistake else 1 - mistake_prob
    correct = pred['example']['mistake'] == predicted_mistake
    
    if not correct and conf >= 0.95:
        output_indices_to_visualize.append(pred_idx)
        
print(f"Identified {len(output_indices_to_visualize)} high-confidence incorrect examples to look at.")
    
random.shuffle(output_indices_to_visualize)
output_indices_to_visualize = output_indices_to_visualize[:20] # randomly pick 20 to look at

100%|██████████| 1000/1000 [00:00<00:00, 28041.29it/s]


Identified 105 high-confidence incorrect examples to look at.


In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image
from pprint import pprint

from travel.data.ego4d import Ego4DMistakeDetectionDataset
from travel.data.mistake_detection import MistakeDetectionExample, get_cutoff_time_by_proportion
from travel.model.vqa import VQAResponse
from travel.model.mistake_detection import aggregate_mistake_probs_over_frames
from travel.model.mistake_detection import DETECTION_FRAMES_PROPORTION

show_frames = True

# Gather up example dirs by example ID
dataset_cache_dir = Ego4DMistakeDetectionDataset.get_cache_dir("val",
                                                         mismatch_augmentation=True,
                                                         multi_frame=True,
                                                         debug_n_examples_per_class=500)
dataset_example_dirs = json.load(open(os.path.join(dataset_cache_dir, "dataset.json"), "r"))["example_dirs"]
dataset_example_dirs = {"/".join(d.split("/")[-3:]): d for d in dataset_example_dirs}

keys = list(preds.keys())
for output_idx in output_indices_to_visualize:
    print(f"Displaying output index {output_idx}\n")
    pred = preds[keys[output_idx]]

    procedure_description = pred['example']['procedure_description']
    mistake = pred["example"]["mistake"]
        
    example_id = pred['example']['example_id']
    example = Ego4DMistakeDetectionDataset.load_example_from_file(dataset_example_dirs[example_id])
    example.cutoff_to_last_frames(DETECTION_FRAMES_PROPORTION)
    pprint(example)
    assert len(example.frames) == len(pred['vqa'])

    # Extract the vqa list - just take the questions and answers for last frame
    # Create a figure with a subplot for each entry
    num_entries = len(pred['vqa'])
    num_questions = len(pred['vqa'][0])
    if show_frames:
        fig, axs = plt.subplots(num_entries, num_questions, figsize=(10 * num_questions, 5 * num_entries))
    print(f"{len(pred['vqa'])} frames")
    for frame_idx in range(len(pred['vqa'])):
        vqa_list = pred['vqa'][frame_idx]

        for i, entry in enumerate(vqa_list):
            if show_frames:
                ax = axs[frame_idx, i] if num_questions > 1 else axs[frame_idx]
                img = Image.open(entry["frame"]) if "frame" in entry and entry["frame"] != "" else example.frames[frame_idx]
                ax.imshow(img)
                ax.axis('off')

            question = entry['question']
            prompt = entry['prompt'].split("USER: <image>")[1].split("ASSISTANT:")[0].strip()
            predicted_answer = entry['predicted_answer']
            expected_answer = entry['expected_answer']

            title = f"Procedure: {procedure_description} ({'mistake' if mistake else 'success'}: {pred['example']['mistake_type']})\n"
            title += f"Q: {question} ({round(example.frame_times[frame_idx], 1)} sec.)\n"
            title += f"Transformed Q: {prompt}\n"
            if i < len(pred['mistake_detection']['0.0']['mistake_probs'][frame_idx]):
                mistake_prob = round(pred['mistake_detection']['0.0']['mistake_probs'][frame_idx][i], 3)
            else:
                mistake_prob = 1.0
            title += f"Predicted Answer: {VQAResponse(predicted_answer).name}, Expected Answer: {VQAResponse(expected_answer).name} ({mistake_prob if mistake_prob == 1.0 or predicted_answer != expected_answer else round(1 - mistake_prob, 3)})"

            if show_frames:
                ax.set_title(title)

    if show_frames:
        plt.tight_layout()
        plt.show()

    print("VQA outputs:")
    for frame_idx in range(len(pred['vqa'])):
        for entry in pred['vqa'][frame_idx]:
            print(entry["question"])
            pprint(entry["answer_probs"])
        print("")
        
    # Print out probabilities
    for k in ["mistake_probs", "nli_relevance_probs", "nli_mistake_probs", "nli_final_mistake_probs"]:
        print(k + ":")
        pprint(pred['mistake_detection']["0.0"][k])
        
    frame_times = pred['example']['frame_times']
    cutoff_time = get_cutoff_time_by_proportion(frame_times, DETECTION_FRAMES_PROPORTION)
    frame_times = [t for t in frame_times if t >= cutoff_time]

    print("\nMISTAKE CONFIDENCE AGGREGATION:")
    mistake_prob = aggregate_mistake_probs_over_frames(pred['mistake_detection']["0.0"]['mistake_probs'], frame_times, verbose=True)
    # print(mistake_prob)


In [8]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from PIL import Image
from pprint import pprint
from tqdm import tqdm

from travel.data.ego4d import Ego4DMistakeDetectionDataset
from travel.data.mistake_detection import MistakeDetectionExample, get_cutoff_time_by_proportion
from travel.model.vqa import VQAResponse
from travel.model.mistake_detection import aggregate_mistake_probs_over_frames
from travel.model.mistake_detection import DETECTION_FRAMES_PROPORTION

show_frames = True

# Gather up example dirs by example ID
dataset_cache_dir = Ego4DMistakeDetectionDataset.get_cache_dir("val",
                                                         mismatch_augmentation=True,
                                                         multi_frame=True,
                                                         debug_n_examples_per_class=250)
dataset_example_dirs = json.load(open(os.path.join(dataset_cache_dir, "dataset.json"), "r"))["example_dirs"]
dataset_example_dirs = {"/".join(d.split("/")[-3:]): d for d in dataset_example_dirs}

keys = list(preds.keys())

n_cropped_frames = 0
n_masked_frames = 0
n_total_frames = 0
for pred in tqdm(preds.values()):
    procedure_description = pred['example']['procedure_description']
    mistake = pred["example"]["mistake"]
        
    example_id = pred['example']['example_id']
    example = Ego4DMistakeDetectionDataset.load_example_from_file(dataset_example_dirs[example_id])
    example.cutoff_to_last_frames(DETECTION_FRAMES_PROPORTION)
    assert len(example.frames) == len(pred['vqa'])

    for frame_idx in range(len(pred['vqa'])):
        vqa_list = pred['vqa'][frame_idx]

        for i, entry in enumerate(vqa_list):
            img = Image.open(entry["frame"]) if "frame" in entry and entry["frame"] != "" else example.frames[frame_idx]
                        
            n_total_frames += 1
            
            # Check the image size to see if it has been cropped by a visual filter
            if img.size != (400, 300):
                n_cropped_frames += 1
                
            # Count number of black pixels to detect whether masking has been applied by a visual filter
            black_pixels = np.asarray(img)
            black_pixels = np.reshape(black_pixels, (-1, 3))
            black_pixels = np.sum(black_pixels, axis=-1)
            n_pixels = len(black_pixels)
            n_black_pixels = n_pixels - np.count_nonzero(black_pixels)
            
            if n_black_pixels > 50:
                n_masked_frames += 1
            
print(f"Cropped frames: {n_cropped_frames}/{n_total_frames} ({n_cropped_frames / n_total_frames})")
print(f"Masked frames: {n_masked_frames}/{n_total_frames} ({n_masked_frames / n_total_frames})")

100%|██████████| 500/500 [00:33<00:00, 14.88it/s]

Cropped frames: 2436/4972 (0.4899436846339501)
Masked frames: 1120/4972 (0.2252614641995173)
